# Sprint 5: Evaluación, Optimización y Despliegue

**Objetivo:** Este notebook cubre la fase final de entrenamiento y optimización del modelo. A diferencia de las fases anteriores, aquí nos enfocamos en refinar el modelo seleccionado (Boosting), optimizar su umbral de decisión para maximizar el F2-Score (dando prioridad a la sensibilidad/recall) y preparar los artefactos finales para el despliegue.

## Pasos a realizar:
1.  **Carga y Preparación de Datos:** Ingesta de datos procesados.
2.  **Configuración del Experimento (PyCaret):** Setup con manejo de desbalance.
3.  **Entrenamiento y Selección:** Comparación de modelos (foco en XGBoost/LGBM).
4.  **Optimización de Hiperparámetros:** Tuning del modelo para mejorar Recall.
5.  **Análisis de Umbral (Threshold):** Búsqueda del punto de corte óptimo.
6.  **Finalización y Persistencia:** Entrenamiento con todos los datos y guardado.
7.  **Simulación de Inferencia:** Prueba rápida del pipeline guardado.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import pickle

# PyCaret
from pycaret.classification import setup, compare_models, tune_model, finalize_model, save_model, load_model, predict_model, pull, get_config
from sklearn.metrics import fbeta_score, recall_score, precision_score, confusion_matrix, classification_report

# Configuración de visualización
%matplotlib inline
sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)

## 1. Carga de Datos
Cargamos el dataset procesado que generamos en el Sprint 4 (`processed_data.parquet`). Este dataset ya tiene las variables codificadas y limpias.

In [ ]:
data_path = "../data/02_intermediate/processed_data.parquet"

if os.path.exists(data_path):
    df = pd.read_parquet(data_path)
    print(f"Data shape: {df.shape}")
else:
    # Fallback por si se corre desde otro directorio
    data_path = "data/02_intermediate/processed_data.parquet"
    if os.path.exists(data_path):
        df = pd.read_parquet(data_path)
        print(f"Data shape: {df.shape}")
    else:
        raise FileNotFoundError("No se encuentra el archivo de datos processed_data.parquet")

# Muestreo para evitar tiempos excesivos en entorno de desarrollo/sandbox
# En producción usaríamos todo el dataset.
MAX_ROWS = 10000 
if len(df) > MAX_ROWS:
    print(f"\u26a0\ufe0f Dataset grande detectado. Muestreando {MAX_ROWS} filas para demostración.")
    df = df.sample(n=MAX_ROWS, random_state=42).reset_index(drop=True)

# Identificar target
target_col = 'CVDINFR4'
if target_col not in df.columns:
    if 'CVDCRHD4' in df.columns:
        target_col = 'CVDCRHD4'
        print(f"Usando target alternativo: {target_col}")
    else:
        raise ValueError("No se encuentra la columna objetivo (CVDINFR4 o CVDCRHD4)")

print(f"Target: {target_col}")
display(df.head())

## 2. Configuración del Experimento (Setup)
Iniciamos PyCaret. Usamos `fix_imbalance=True` para aplicar SMOTE automáticamente en el set de entrenamiento, ya que sabemos que la clase positiva (enfermedad cardíaca) es minoritaria.

In [ ]:
exp = setup(
    data=df,
    target=target_col,
    session_id=42,
    fix_imbalance=True, # Importante para clases desbalanceadas
    verbose=False,
    html=True
)

print("Setup completado. Configuración:")
print(f"Train shape: {get_config('X_train').shape}")
print(f"Test shape: {get_config('X_test').shape}")

## 3. Comparación y Selección de Modelos
Comparamos varios modelos, pero nos enfocamos en los basados en árboles (Gradient Boosting) que suelen funcionar mejor para datos tabulares. Ordenamos por `Recall` porque nos interesa minimizar los Falsos Negativos.

In [ ]:
# Limitamos a modelos rápidos y potentes para la demo
best_model = compare_models(
    include=['xgboost', 'lightgbm', 'rf', 'gbc'],
    sort='Recall',
    n_select=1
)

print(f"Mejor modelo seleccionado: {best_model}")

## 4. Optimización de Hiperparámetros (Tuning)
Buscamos mejorar el rendimiento del modelo seleccionado. Optimizamos explícitamente para `Recall`.

In [ ]:
print("Optimizando hiperparámetros...")
tuned_model = tune_model(best_model, optimize='Recall', n_iter=20)

print("Modelo optimizado:")
print(tuned_model)

## 5. Análisis de Umbral (Threshold Optimization)
El umbral por defecto es 0.5. Sin embargo, para maximizar el F2-Score (que valora el Recall sobre la Precision), necesitamos mover este umbral. Calculamos las métricas para distintos umbrales y elegimos el óptimo.

In [ ]:
# Predecir probabilidades en el set de prueba (hold-out)
predictions = predict_model(tuned_model, raw_score=True)

# Extraer etiquetas reales y scores
y_true = predictions[target_col]

# PyCaret nombra las columnas de score como prediction_score_1 o Score_1
score_cols = [c for c in predictions.columns if 'score' in c.lower() or 'prob' in c.lower()]
# Buscamos la columna de la clase positiva (generalmente 1)
possible_score_cols = ['prediction_score_1', 'Score_1', 'proba_1']
score_col = next((c for c in possible_score_cols if c in predictions.columns), None)

if score_col:
    y_scores = predictions[score_col]
    
    thresholds = np.arange(0, 1, 0.01)
    f2_scores = []
    recalls = []
    precisions = []

    for t in thresholds:
        y_pred_t = (y_scores >= t).astype(int)
        f2_scores.append(fbeta_score(y_true, y_pred_t, beta=2))
        recalls.append(recall_score(y_true, y_pred_t))
        precisions.append(precision_score(y_true, y_pred_t, zero_division=0))

    # Encontrar el mejor umbral para F2
    best_idx = np.argmax(f2_scores)
    best_thresh = thresholds[best_idx]
    best_f2 = f2_scores[best_idx]
    
    print(f"\n\U0001f3af Mejor Umbral (F2-Score): {best_thresh:.2f}")
    print(f"F2-Score: {best_f2:.4f}")
    print(f"Recall: {recalls[best_idx]:.4f}")
    print(f"Precision: {precisions[best_idx]:.4f}")
    
    # Visualización
    plt.figure(figsize=(10, 6))
    plt.plot(thresholds, f2_scores, label='F2 Score', color='green', linewidth=2)
    plt.plot(thresholds, recalls, label='Recall', color='blue', linestyle='--')
    plt.plot(thresholds, precisions, label='Precision', color='red', linestyle=':')
    plt.axvline(best_thresh, color='black', linestyle='-.', label=f'Optimum ({best_thresh})')
    plt.title('Métricas vs Umbral de Decisión')
    plt.xlabel('Umbral')
    plt.ylabel('Score')
    plt.legend()
    plt.show()

else:
    print("No se encontró columna de probabilidad. Se usará umbral por defecto 0.5")
    best_thresh = 0.5

## 6. Finalización y Persistencia (Model Finalization)
Una vez satisfechos con los hiperparámetros y el umbral, entrenamos el modelo con **todos** los datos disponibles (Train + Test) para maximizar la información aprendida antes del despliegue.

In [ ]:
print("Finalizando modelo (entrenando con dataset completo)...")
final_model = finalize_model(tuned_model)

# Guardar Artefactos
output_dir = "../models"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1. Guardar Pipeline (.pkl)
model_name = "final_pipeline_v1"
save_path = os.path.join(output_dir, model_name)
save_model(final_model, save_path)

# 2. Guardar Configuración (Umbral)
config_data = {
    "threshold": float(best_thresh),
    "model_name": str(best_model.__class__.__name__),
    "metric_optimized": "F2-Score"
}

config_path = os.path.join(output_dir, "model_config.json")
with open(config_path, 'w') as f:
    json.dump(config_data, f, indent=4)

print(f"\n\u2705 Modelo guardado en: {save_path}.pkl")
print(f"\u2705 Configuración guardada en: {config_path}")

## 7. Simulación de Uso (Inferencia)
Demostramos cómo la aplicación (Streamlit) usaría estos archivos para hacer predicciones.

In [ ]:
# Cargar pipeline
loaded_pipeline = load_model(save_path)

# Cargar config
with open(config_path, 'r') as f:
    loaded_config = json.load(f)
thresh = loaded_config['threshold']

print(f"Pipeline y umbral ({thresh}) cargados.")

# Tomar un ejemplo aleatorio del dataset original
sample_data = df.sample(1)
print("\nDatos de entrada (ejemplo):")
display(sample_data.drop(columns=[target_col]))

# Predecir Probabilidad
pred_raw = predict_model(loaded_pipeline, data=sample_data, raw_score=True)

# Lógica de decisión personalizada
score_col_final = [c for c in pred_raw.columns if 'score' in c.lower() and '1' in c][0]
prob = pred_raw[score_col_final].values[0]

decision = "ALTO RIESGO" if prob >= thresh else "BAJO RIESGO"

print(f"\nProbabilidad calculada: {prob:.4f}")
print(f"Umbral de corte: {thresh:.4f}")
print(f"Resultado Final: {decision}")